<a href="https://colab.research.google.com/github/Rahul19982022/anime-recommendation-system-exp/blob/main/archives/1_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pandas==2.2.2
!pip install -q numpy==2.0.2

In [2]:
import pandas as pd
import numpy as np
import os
import pickle

In [3]:
pwd

'/content'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# put path of your project folder here
proj_path = '/content/drive/MyDrive/anime_recommendation_system'
os.chdir(proj_path)

In [6]:
%%time
user_data = pd.read_csv('datasets/kaggle_dataset/animelist.csv')

CPU times: user 35.3 s, sys: 12.3 s, total: 47.6 s
Wall time: 1min 4s


In [ ]:
anime = pd.read_csv('datasets/kaggle_dataset/anime.csv')
watching_status = pd.read_csv('datasets/kaggle_dataset/watching_status.csv')

In [ ]:
watching_status

,status,description
0,1,Currently Watching
1,2,Completed
2,3,On Hold
3,4,Dropped
4,6,Plan to Watch


In [ ]:
user_data.head()

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


In [ ]:
anime.head(2)

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0


In [ ]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17562 entries, 0 to 17561
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MAL_ID         17562 non-null  int64 
 1   Name           17562 non-null  object
 2   Score          17562 non-null  object
 3   Genres         17562 non-null  object
 4   English name   17562 non-null  object
 5   Japanese name  17562 non-null  object
 6   Type           17562 non-null  object
 7   Episodes       17562 non-null  object
 8   Aired          17562 non-null  object
 9   Premiered      17562 non-null  object
 10  Producers      17562 non-null  object
 11  Licensors      17562 non-null  object
 12  Studios        17562 non-null  object
 13  Source         17562 non-null  object
 14  Duration       17562 non-null  object
 15  Rating         17562 non-null  object
 16  Ranked         17562 non-null  object
 17  Popularity     17562 non-null  int64 
 18  Members        17562 non-n

In [ ]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109224747 entries, 0 to 109224746
Data columns (total 5 columns):
 #   Column            Dtype
---  ------            -----
 0   user_id           int64
 1   anime_id          int64
 2   rating            int64
 3   watching_status   int64
 4   watched_episodes  int64
dtypes: int64(5)
memory usage: 4.1 GB


In [ ]:
user_data.isna().sum()

,0
user_id,0
anime_id,0
rating,0
watching_status,0
watched_episodes,0


In [ ]:
user_data.max()

,0
user_id,353404
anime_id,48492
rating,10
watching_status,55
watched_episodes,65535


In [ ]:
import gc
gc.collect()

41

In [ ]:
user_data['rating'] = user_data['rating'].astype(np.uint8)
user_data['user_id'] = user_data['user_id'].astype(np.uint32)
user_data['anime_id'] = user_data['anime_id'].astype(np.uint16)

In [ ]:
gc.collect()

0

In [ ]:
user_data['watching_status'].value_counts()

,count
watching_status,
2,68089751
6,27938693
1,5228658
4,4266591
3,3700514
0,531
5,6
33,2
55,1


In [ ]:
watching_status

,status,description
0,1,Currently Watching
1,2,Completed
2,3,On Hold
3,4,Dropped
4,6,Plan to Watch


In [ ]:
anime['episodes'] = anime['Episodes'][anime['Episodes'].str.isnumeric()].astype(int)
anime[anime['episodes'].isna()]['Episodes'].value_counts()

,count
Episodes,
Unknown,516


In [ ]:
user_data['anime_episodes'] = (user_data['anime_id'].map(anime.set_index('MAL_ID')['episodes'])).astype('float32')

In [ ]:
cond1 = (~user_data['watching_status'].isin(pd.Series(watching_status['status'].unique().tolist())))
cond2 = ((user_data['watched_episodes'] == 0))
cond3 = (user_data['watched_episodes'] == user_data['anime_episodes'])

user_data.loc[cond1 & cond2, 'watching_status'] = 6
user_data.loc[cond1 & cond3, 'watching_status'] = 2
user_data.loc[cond1, 'watching_status'] = 1

In [ ]:
user_data['watching_status'].value_counts().sort_index()

,count
watching_status,
1,5229198
2,68089751
3,3700514
4,4266591
6,27938693


In [ ]:
gc.collect()

30

In [ ]:
user_data['watched_episodes'] = user_data['watched_episodes'].astype(np.uint16)

In [ ]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109224747 entries, 0 to 109224746
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           uint32 
 1   anime_id          uint16 
 2   rating            uint8  
 3   watching_status   int64  
 4   watched_episodes  uint16 
 5   anime_episodes    float32
dtypes: float32(1), int64(1), uint16(2), uint32(1), uint8(1)
memory usage: 2.1 GB


In [ ]:
user_data.loc[(user_data['watched_episodes'] > user_data['anime_episodes']), 'watched_episodes'] = \
user_data.loc[(user_data['watched_episodes'] > user_data['anime_episodes']), 'anime_episodes']

In [ ]:
user_data.head()

,user_id,anime_id,rating,watching_status,watched_episodes,anime_episodes
0,0,67,9,1,1,24.0
1,0,6702,7,1,4,175.0
2,0,242,10,1,4,13.0
3,0,4898,0,1,1,24.0
4,0,21,10,1,0,NaN


In [ ]:
user_data = user_data.drop(['anime_episodes'], axis = 1)

In [ ]:
user_data.head()

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


In [ ]:
user_data['rating'].value_counts().sort_index(ascending=False)

,count
rating,
10,7144392
9,10235934
8,15422150
7,14244633
6,7543377
5,4029645
4,1845854
3,905700
2,545339


In [ ]:
user_data = user_data[(user_data['rating'] != 0) & (user_data['watched_episodes'] != 0)]
user_data.reset_index(drop = True, inplace = True)

In [ ]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61699446 entries, 0 to 61699445
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           uint32
 1   anime_id          uint16
 2   rating            uint8 
 3   watching_status   int64 
 4   watched_episodes  uint16
dtypes: int64(1), uint16(2), uint32(1), uint8(1)
memory usage: 1000.3 MB


In [ ]:
user_data.head()

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,24,9,1,5
4,0,4722,8,1,4


In [ ]:
user_data['anime_id'].nunique()

17156

In [ ]:
user_data.to_pickle('datasets/created_datasets/user_rated_data_Step1.pkl')